In [1]:
import pandas as pd
from sklearn import tree, naive_bayes, svm
from sklearn import neural_network
import ast
import math
import pickle
import os
import glob

Make all the constants from the game available here to use.

In [2]:
#[General_Constants]
FIELD_WIDTH = 19
FIELD_HEIGHT = 15

#[Game_Constants]
NO_ITERATIONS = 100
MAX_CALC_TIME = 100000

#[Field_Constants]
CELL_EMPTY = '.'
CELL_SHEEP_1 = 'S'
CELL_SHEEP_1_d = 'U'
CELL_WOLF_1 = 'W'
CELL_SHEEP_2 = 's'
CELL_SHEEP_2_d = 'u'
CELL_WOLF_2 = 'w'
CELL_GRASS = 'g'
CELL_RHUBARB = 'r'
CELL_FENCE = '#'


#[Movements]
MOVE_NONE = 0
MOVE_UP = -1
MOVE_DOWN = 1
MOVE_LEFT = -2
MOVE_RIGHT = 2

#[Awards]
AWARD_RHUBARB = 5
AWARD_GRASS = 1

# Load data

Load all games into one dataframe

In [3]:
path = "../training_data/subset2" 
all_files = glob.glob(path + "/*.csv")   

training_data = []

#load the data into a pandas frames
for file in all_files:
    game_data = pd.read_csv(file,index_col=False)
    reason = game_data.iloc[-1][6]
    
    #if the reason is found, add it to each line to fill out the blanks
    if type(reason) is str:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = reason

    #else there was no reason, implying the game reached the number of iterations
    else:
        for index,row in game_data.iterrows():
            game_data.loc[index,'reason'] = 'max_iterations'    
    
    training_data.append(game_data)

#preview the final 5 lines
training_data[-1].head()

,field_before,field_after,turn_made_by,move_made,score1,score2,reason
0,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,2,0,0,max_iterations
1,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-2,0,0,max_iterations
2,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 sheep,1,0,0,max_iterations
3,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player2 sheep,-1,0,0,max_iterations
4,"[['.', '.', '.', '.', '.', '.', '.', '.', '.',...","[['.', '.', '.', '.', '.', '.', '.', '.', '.',...",player1 wolf,2,0,0,max_iterations


# Feature selection and Instance selection 

### General Note: 
I imported the class of my Python file to use the same functions and prevent duplicating code. I have to use the same feature preparation in the training phase as in the prediction pmhase. Thus all feature engineering is done in the python file (this also allows for easier debugging).

## Sheep

### Feature Selection:
My baseline was the Search Algorithm from assignment 1. I used the movement determined by the search algorithm as a feature for my sheep. I then tried multiple features and looked at their correlation to the actual move made. I dropped features with poor correlation. Only using the results form assignment 1 yieled the best results. This is basically the same feature selection as in assignment 2, I just optimized the hyperparameters of the MLPClassifier.

### Data Selection
I used all rows from the winning side performing a sheep-move. I used different configurations of agents from assignment 1 and ran them on random maps to scrape the results. I didn't use the provided data, as I didn't know the quality of the games, as the second worst agents still wins against the worst agent resulting it to be a "win" although both agents are in fact not optimal. I also ignored all games that ended because of an exception.

In [4]:
from importlib import reload  # to get changes in code
import chriweb_a3
chriweb_a3 = reload(chriweb_a3)
ii = chriweb_a3.IntrepidIbex()

X_sheep = []
Y_sheep = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep_label = 's'
        wolf_label = 'W'
        figure = 1
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep_label = 'S'
        wolf_label = 'w'
        figure = 2
    else:
        continue
        
    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] not in ('player1 sheep','player2 sheep'):
            continue
            
        #we want to only learn from winning player
        if str(figure) not in row['turn_made_by']:
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field = ast.literal_eval(row['field_before'])     
                
        game_features = ii.get_features_sheep(figure, field)  
        
        #add features and move to X_sheep and Y_sheep
        X_sheep.append(game_features)
        Y_sheep.append(move)  

In [5]:
df_investigate_s = pd.DataFrame(X_sheep)
df_investigate_s['y'] = Y_sheep
df_investigate_s

,0,y
0,-2,-1
1,-2,2
2,2,2
3,2,2
4,-1,-1
5,-1,-1
6,1,1
7,2,2
8,2,2
9,-1,1


See correlation of each feature to result. I dropped features that had not enough correlation.

In [6]:
df_investigate_s.corr()['y']

0    0.563276
y    1.000000
Name: y, dtype: float64

## Wolf

### Feature Selection:
My baseline was the Search Algorithm from assignment 1. I used the movement determined by the search algorithm as a feature for my wolf. I then tried multiple features and looked at their correlation to the actual move made. I dropped features with poor correlation. This is basically the same as in assignment 2, I optimized the hyperparameters for the MLPClassifier.

### Data Selection
I used all rows from the winning side performing a wolf-move. I used different configurations of agents from assignment 1 and ran them on random maps to scrape the results. I didn't use the provided data, as I didn't know the quality of the games, as the second worst agents still wins against the worst agent resulting it to be a "win" although both agents are in fact not optimal. I also ignored all games that ended because of an exception.

In [7]:
from importlib import reload  # to get changes in code
import chriweb_a3
chriweb_a3 = reload(chriweb_a3)
ii = chriweb_a3.IntrepidIbex()

X_wolf = []
Y_wolf = []
number_moves = 0

for game in training_data:
    
    #we want to learn from the winning player, which is the player with the highest score:
    if game.iloc[-1][4] < game.iloc[-1][5]:
        sheep_label = 's'
        wolf_label = 'W'
        figure = 1
    
    elif game.iloc[-1][4] > game.iloc[-1][5]:
        sheep_label = 'S'
        wolf_label = 'w'
        figure = 2
    else:
        continue

    rhubarb = 'r'
    grass = 'g'

    #for each game state in our training data
    for index,row in game.iterrows():

        #we don't want games that ended because of an error or because the sheep commited suicide
        if row['reason'] not in ('sheep1 eaten','sheep2 eaten','max_iterations'):
            continue

        #we want to only learn from sheep
        if row['turn_made_by'] not in ('player1 wolf','player2 wolf'):
            continue
            
        #we want to only learn from winning player
        if str(figure) not in row['turn_made_by']:
            continue
        
        number_moves += 1
        
        #this is the move that we are learning from this game state
        move = row['move_made']

        #create empty feature array for this game state
        game_features = []

        #turn the field from before the move from a string back to a list
        field = ast.literal_eval(row['field_before'])     
        
        game_features = ii.get_features_wolf(figure, field)  
        
        #add features and move to X_sheep and Y_sheep
        X_wolf.append(game_features)
        Y_wolf.append(move)  

In [8]:
df_investigate_w = pd.DataFrame(X_wolf)
df_investigate_w['y'] = Y_wolf
df_investigate_w

,0,y
0,-2,-2
1,-1,-2
2,-2,-1
3,-2,-1
4,-2,-2
5,-2,-1
6,-2,-2
7,-2,-2
8,-2,-1
9,1,2


In [9]:
df_investigate_w.corr()['y']

0    0.794627
y    1.000000
Name: y, dtype: float64

# Build Sheep Model

Split into train and test data to measure performance

In [10]:
from sklearn import model_selection
x_train_s, x_test_s, y_train_s, y_test_s = model_selection.train_test_split(X_sheep, Y_sheep, test_size=0.25)


## Train & Test sheep

Allowed algorithms:
* MLPClassifier

In [11]:
from sklearn.metrics import confusion_matrix
import numpy as np

##  MLPClassifier

In [12]:
sheep_nn = neural_network.MLPClassifier(hidden_layer_sizes=(35,), activation='tanh', random_state=1,verbose=False)
sheep_nn = sheep_nn.fit(x_train_s,y_train_s)
y_pred_s = sheep_nn.predict(x_test_s)

cm = confusion_matrix(y_test_s, y_pred_s.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

[[36 14  0  3  7]
 [ 6 21  0  4  6]
 [ 0  1  0  0  0]
 [ 3  7  0 30  5]
 [ 8  5  0  4 23]]
0.48184684684684687
nan


/home/chris/PAI/PAI_Ex3/venv/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/chris/PAI/PAI_Ex3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


# Train wolf

## MLPClassifier

In [13]:
from sklearn import model_selection
x_train_w, x_test_w, y_train_w, y_test_w = model_selection.train_test_split(X_wolf, Y_wolf, test_size=0.25)

In [14]:
wolf_nn = neural_network.MLPClassifier(hidden_layer_sizes=(35,), activation='tanh', random_state=1)
wolf_nn = wolf_nn.fit(x_train_w,y_train_w)
y_pred_w = wolf_nn.predict(x_test_w)

cm = confusion_matrix(y_test_w, y_pred_w.tolist())
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)

print(cm)
print(np.mean(recall))
print(np.mean(precision))

[[22  2  0  0]
 [ 5 19  0  0]
 [ 3  0 19  1]
 [ 0  1  0 19]]
0.8711050724637681
0.8867424242424242


/home/chris/PAI/PAI_Ex3/venv/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Save models to files


Save your models to files here using pickle. Change the [uzhshortname] to your own UZH shortname. This name needs to match the model that you caller in your python player file.

In [15]:
sheep_filename = 'chriweb_sheep_model.sav'
wolf_filename = 'chriweb_wolf_model.sav'

pickle.dump(sheep_nn,open(sheep_filename,'wb'))
pickle.dump(wolf_nn,open(wolf_filename,'wb'))